In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
#from sklearn.metrics import classification_report, confusion_matrix
from pyspark.mllib.evaluation import MulticlassMetrics


import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("fa21-ds5110-group10") \
    .getOrCreate()

First we add in our cached dataset from our prior feature engineering.

In [3]:
df = spark.read.parquet("../../../data/processed/chess_games_moves_model.parquet")

In [4]:
df.printSchema()

root
 |-- event: string (nullable = true)
 |-- white_result: string (nullable = true)
 |-- WhiteElo: integer (nullable = true)
 |-- BlackElo: integer (nullable = true)
 |-- first_two: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ECO: string (nullable = true)
 |-- Opening: string (nullable = true)
 |-- game_complexity: integer (nullable = true)



Next we review the resulting data points of interest.
We notice that ECO and the first two sets of moves are distinct of one another, and may influence the overall model's prediction.

In [5]:
print("First Turns: {}".format(df.select("first_two").distinct().count()))
print("ECO Types: {}".format(df.select("ECO").distinct().count()))
print("Opening Variants: {}".format(df.select("Opening").distinct().count()))
print( df.groupBy('first_two').count().sort(F.col("count").desc()).show(5), df.count())

First Turns: 464628
ECO Types: 493
Opening Variants: 2897
+------------------+------+
|         first_two| count|
+------------------+------+
|  [e4 e5, Nf3 Nc6]|332412|
|   [e4 e5, Nf3 d6]|129785|
|[e4 d5, exd5 Qxd5]| 86575|
|  [e4 c5, Nf3 Nc6]| 78638|
|    [e4 e6, d4 d5]| 72122|
+------------------+------+
only showing top 5 rows

None 3849646


We will now begin to build a model, keying in on the opening move and the white_result columns.
Note that both of these are categorical values, so we will need to encode them using the StringIndexer for pyspark to do model evaluations.

In [6]:
opening_vectorizor = StringIndexer(inputCol="ECO", outputCol="opening_ohe")
opening_dataframe = opening_vectorizor.fit(df).transform(df)

gametype_vectorizer = StringIndexer(inputCol="event", outputCol="event_vector")
opening_dataframe = gametype_vectorizer.fit(opening_dataframe).transform(opening_dataframe)

result_vectorizor = StringIndexer(inputCol="white_result", outputCol="white_result_vector")
opening_dataframe = result_vectorizor.fit(opening_dataframe).transform(opening_dataframe)

In [7]:
opening_dataframe.show(3)

+---------+------------+--------+--------+----------------+---+--------------------+---------------+-----------+------------+-------------------+
|    event|white_result|WhiteElo|BlackElo|       first_two|ECO|             Opening|game_complexity|opening_ohe|event_vector|white_result_vector|
+---------+------------+--------+--------+----------------+---+--------------------+---------------+-----------+------------+-------------------+
|    Blitz|         win|    2068|    1846|[c4 c5, Nc3 Nf6]|A34|English Opening: ...|              6|      133.0|         0.0|                0.0|
|    Blitz|         win|    1708|    1399| [d4 b6, c4 Bb7]|A40|  English Defense #2|              5|        6.0|         0.0|                0.0|
|Classical|        loss|    1542|    1790| [e4 Nc6, d4 d5]|B00|Nimzowitsch Defen...|              7|        7.0|         1.0|                1.0|
+---------+------------+--------+--------+----------------+---+--------------------+---------------+-----------+------------

Next we perform One-Hot Encoding on our Opening type (or ECO) and do our comparision.  THis will create a new column that we will use for our random forest model.

In [8]:
from pyspark.ml.feature import OneHotEncoder


encoder = OneHotEncoder(inputCols=["opening_ohe"],
                        outputCols=["ECO_Type"])
model = encoder.fit(opening_dataframe)
encoded = model.transform(opening_dataframe)

encoded.show()

+---------+------------+--------+--------+--------------------+---+--------------------+---------------+-----------+------------+-------------------+-----------------+
|    event|white_result|WhiteElo|BlackElo|           first_two|ECO|             Opening|game_complexity|opening_ohe|event_vector|white_result_vector|         ECO_Type|
+---------+------------+--------+--------+--------------------+---+--------------------+---------------+-----------+------------+-------------------+-----------------+
|    Blitz|         win|    2068|    1846|    [c4 c5, Nc3 Nf6]|A34|English Opening: ...|              6|      133.0|         0.0|                0.0|(492,[133],[1.0])|
|    Blitz|         win|    1708|    1399|     [d4 b6, c4 Bb7]|A40|  English Defense #2|              5|        6.0|         0.0|                0.0|  (492,[6],[1.0])|
|Classical|        loss|    1542|    1790|     [e4 Nc6, d4 d5]|B00|Nimzowitsch Defen...|              7|        7.0|         1.0|                1.0|  (492,[7],

Now that we have the OHE of our ECO, we can combine it with other features to build out our predictors for random forest.

In [9]:
features_assembler = VectorAssembler(inputCols=['ECO_Type', "event_vector"], outputCol='features')
encoded = features_assembler.transform(encoded)

Now that our data is model-ready, we will do a split, fit, transform, and evaluation to determine the performance of our model.
Note that we have chosen the default tunings, but in the future we will likely apply a cross-validation technique in pyspark to select the correct hyperparameters.

In [10]:
trainData, testData, validationData  = encoded.randomSplit([0.6, 0.3, 0.1], seed=314)

In [11]:
rf = RandomForestClassifier(numTrees=20, maxDepth=5, featuresCol='features', labelCol='white_result_vector', seed=1337, leafCol="leafId")
rf_model = rf.fit(trainData)
rf_result = rf_model.transform(testData)

In [12]:
accuracy_evaluator = BinaryClassificationEvaluator(labelCol='white_result_vector', metricName='areaUnderROC')
model_accuracy = accuracy_evaluator.evaluate(rf_result)

In [13]:
print("Model Accuracy: {}".format(model_accuracy))

Model Accuracy: 0.524294869672513


Here we see that our model's performance seems to work well, with roughtly 50% of games resulting in a match to white win.

Let's review our classifications and confusion matrix next to determine the overall performance.

In [14]:
designer_matrix = rf_result.select(['prediction','white_result_vector'])
metrics_rdd = MulticlassMetrics(designer_matrix.rdd.map(tuple))
print(metrics_rdd.confusionMatrix().toArray())

[[621708.      0.]
 [534408.      0.]]


Reviewing this model, we see that this model is not good.
Of the three classes, we found that the precision for the white loss performs okay, however the white win and tie both evaluate to 0 for precision.
This means that our model is overfitting and failed to correctly identify either ties or losses.

Further tuning will be required to better distribute our data (either adjusting the threshold, or tuning the tree).